In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [64]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True), override=True)

import os
print("OPENAI ok:", bool(os.getenv("OPENAI_API_KEY")))
print("REPLICATE ok:", bool(os.getenv("REPLICATE_API_TOKEN")))

OPENAI ok: True
REPLICATE ok: True


In [65]:
# 기본 임포트
from typing import List, Dict, Literal, TypedDict
from pydantic import BaseModel, Field
from typing_extensions import Annotated

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

In [66]:
# 1) 스키마 정의 (EmotionResult / MusicBrief / GraphState)
Valence = Annotated[float, Field(ge=-1.0, le=1.0)]
Arousal = Annotated[float, Field(ge=0.0, le=1.0)]
BPM     = Annotated[int,   Field(ge=50,  le=140)]
DurSec  = Annotated[int,   Field(ge=60,   le=90)] # 음악 재생시간(60초 ~ 90초)

class EmotionResult(BaseModel):
    primary: str
    valence: Valence = 0.0
    arousal: Arousal = 0.5
    confidence: Arousal = 0.7
    reasons: str = "—"

class MusicBrief(BaseModel):
    mood: str
    bpm: BPM = 90
    key: str
    duration_sec: DurSec = 60               # 음악 기본 재생시간 60초
    instruments: List[str] = []
    style_tags: List[str] = []
    prompt: str  # 영어 프롬프트

class GraphState(TypedDict, total=False):
    user_text: str
    emotion: EmotionResult
    brief: MusicBrief
    audio_path: str
    provider_used: Literal["replicate"]
    meta: Dict


In [67]:
# 2) LLM 헬퍼 (OpenAI)
def get_llm():
    # 스키마 맞춤 출력이 중요 → 낮은 temperature
    return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


In [68]:
# 3) 감정 분석 노드
def analyze_emotion_node(state: GraphState) -> GraphState:
    llm = get_llm()
    sys = (
        "당신은 심리 정서를 요약하는 분석가입니다. "
        "사용자 텍스트에서 주요 감정을 한 단어(또는 짧은 구)로 도출하고, "
        "valence(-1~1), arousal(0~1), confidence(0~1)을 추정하세요. "
        "반드시 EmotionResult(JSON 스키마)에 맞춰 응답하세요."
    )
    structured = llm.with_structured_output(EmotionResult)
    result = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":state["user_text"]}
    ])
    state["emotion"] = result
    return state

In [75]:
# 4) 음악 브리프 노드
def compose_brief_node(state: GraphState) -> GraphState:
    llm = get_llm()
    emo: EmotionResult = state["emotion"]
    sys = (
    "너는 음악 감독이다. 아래 감정 분석과 사용자 텍스트를 참고해 "
    "치유/안정 목적의 짧은 BGM을 위한 Music Brief를 JSON으로 만들어라.\n"
    "대체로 다음 규칙을 따른다:\n"
    "1) bpm: 50~140. arousal이 높을수록 빠르게(대략 60 + arousal*40), 낮을수록 느리게.\n"
    "2) duration_sec: 60~90. valence<=-0.2 또는 arousal>0.6이면 78~90, 그 외 60~78.\n"
    "3) key: valence>=0.2 → 메이저(C/G/F/D major 등), valence<=-0.2 → 마이너(A/D/E/B minor 등), 그 외엔 두 계열 중 선택. "
    "'C major'를 기본값으로 반복 사용하지 말 것.\n"
    "4) instruments: 2~4개. 기본은 warm piano, soft pad 중심. "
    "불안/고각성(arousal>0.6)에는 light percussion/gentle pulse를 소량, "
    "우울(valence<-0.2)에는 strings 또는 soft choir를 권장.\n"
    "5) style_tags: 3~5개 (예: calming, minimal, warm, ambient, breathing).\n"
    "6) prompt: 영어 한 문장, 25단어 이내. 악기/무드/다이내믹/리듬 강도(soft/very soft 등)를 포함하되 "
    "숫자(bpm/duration/key)는 넣지 말 것.\n"
    "7) JSON만 출력. 추가 설명 금지.\n"
    )  

    usr = (
        f"# Emotion\nprimary={emo.primary}, valence={emo.valence}, "
        f"arousal={emo.arousal}, confidence={emo.confidence}\n\n"
        f"# Text\n{state['user_text']}\n"
    )
    structured = llm.with_structured_output(MusicBrief)
    brief = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":usr}
    ])

# ▼ duration 보정 (60~90초로 강제)
    if brief.duration_sec < 60:
        brief = brief.model_copy(update={"duration_sec": 60})
    elif brief.duration_sec > 90:
        brief = brief.model_copy(update={"duration_sec": 90})

    state["brief"] = brief
    return state

In [70]:
# # 5) Replicate(Stable Audio 2.5) 호출 함수
# 상단 import에 추가
from urllib.parse import urlparse

def generate_with_replicate_strict(prompt: str, duration: int) -> str:
    tok = os.getenv("REPLICATE_API_TOKEN")
    assert tok, "REPLICATE_API_TOKEN이 없습니다 (.env 확인)"
    assert 60 <= int(duration) <= 90, f"duration(초)은 60~90 범위여야 합니다: {duration}"

    # 최신 클라이언트는 기본적으로 FileOutput을 반환
    out = replicate.run(MODEL_ID, input={"prompt": prompt, "duration": int(duration)})
    first = out[0] if isinstance(out, (list, tuple)) else out

    os.makedirs("outputs", exist_ok=True)
    ts = int(time.time())

    # 1) FileOutput (권장 경로) - .read()로 바로 저장
    if hasattr(first, "read"):  # file-like (FileOutput)
        # 확장자 추정: URL 있으면 거기서 뽑기
        ext = ".bin"
        url_attr = getattr(first, "url", None)  # FileOutput은 .url을 제공
        if isinstance(url_attr, str):
            ext_candidate = os.path.splitext(urlparse(url_attr).path)[1].lower()
            if ext_candidate:
                ext = ext_candidate
        out_path = f"outputs/stableaudio_{ts}{ext}"
        with open(out_path, "wb") as f:
            f.write(first.read())  # 전체 바이트 저장
        return out_path

    # 2) 과거/옵트아웃 경로: URL 문자열
    if isinstance(first, str):
        r = requests.get(first, timeout=120); r.raise_for_status()
        ct = (r.headers.get("Content-Type") or "").lower()
        # content-type 또는 URL에서 확장자 추정
        if "wav" in ct:
            ext = ".wav"
        elif "mpeg" in ct or "mp3" in ct:
            ext = ".mp3"
        else:
            ext = os.path.splitext(urlparse(first).path)[1] or ".bin"
        out_path = f"outputs/stableaudio_{ts}{ext}"
        with open(out_path, "wb") as f:
            f.write(r.content)
        return out_path

    # 3) 드물게 dict 형태로 오는 경우
    if isinstance(first, dict):
        url = first.get("url") or first.get("audio") or first.get("output")
        if isinstance(url, str):
            r = requests.get(url, timeout=120); r.raise_for_status()
            ext = os.path.splitext(urlparse(url).path)[1] or ".bin"
            out_path = f"outputs/stableaudio_{ts}{ext}"
            with open(out_path, "wb") as f:
                f.write(r.content)
            return out_path

    raise RuntimeError(f"Unexpected replicate output type: {type(first)}")


In [71]:
# 6) 음악 생성 노드 (Replicate만)
def generate_music_node(state: GraphState) -> GraphState:
    brief: MusicBrief = state["brief"]
    path = generate_with_replicate_strict(brief.prompt, int(brief.duration_sec))

    state["audio_path"] = path
    state["provider_used"] = "replicate"
    state["meta"] = {
        "emotion": state["emotion"].model_dump(),
        "brief": state["brief"].model_dump(),
        "provider": "replicate",
        "path": path,
    }
    return state

In [72]:
workflow = StateGraph(GraphState)
workflow.add_node("analyze_emotion", analyze_emotion_node)
workflow.add_node("compose_brief",  compose_brief_node)
workflow.add_node("generate_music", generate_music_node)

workflow.add_edge(START, "analyze_emotion")
workflow.add_edge("analyze_emotion", "compose_brief")
workflow.add_edge("compose_brief",  "generate_music")
workflow.add_edge("generate_music", END)

graph = workflow.compile()

In [74]:
state = {
    "user_text": "시험에 합격해서 하루 종일 들떠. 발걸음이 저절로 빨라져"
}
final = graph.invoke(state)

print("=== Emotion ===")
print(final["emotion"].model_dump())
print("\n=== Music Brief ===")
print(final["brief"].model_dump())
print("\n=== Provider Used ===")
print(final["provider_used"])
print("\n=== Audio Path ===")
print(final["audio_path"])

=== Emotion ===
{'primary': '기쁨', 'valence': 1.0, 'arousal': 0.8, 'confidence': 0.9, 'reasons': '시험 합격이라는 긍정적인 경험으로 인해 기쁜 감정이 표현되고 있으며, 들뜬 상태와 발걸음이 빨라지는 것은 높은 각성을 나타냅니다.'}

=== Music Brief ===
{'mood': 'Joyful', 'bpm': 100, 'key': 'G major', 'duration_sec': 70, 'instruments': ['warm piano', 'soft pad', 'light percussion'], 'style_tags': ['uplifting', 'energetic', 'bright', 'minimal'], 'prompt': 'A bright and uplifting piece featuring warm piano and soft pads, with a gentle pulse to enhance the joyful mood.'}

=== Provider Used ===
replicate

=== Audio Path ===
outputs/stableaudio_1758012347.mp3
